In [ ]:
import os
PROJECT = "tatvic-gcp-dev-team" # REPLACE WITH YOUR PROJECT ID
BUCKET = 'data-etl-process1' # REPLACE WITH YOUR BUCKET NAME
REGION = 'asia-east1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1



In [ ]:
# For Python Code
# Model Info
MODEL_NAME = 'virality'
# Model Version
MODEL_VERSION = 'v1'
# Training Directory name
TRAINING_DIR = 'cloudmlemodels'

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 
os.environ['TFVERSION'] = '1.13'  # Tensorflow version

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

In [ ]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud Machine Learning Engine API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

In [ ]:
#%%bash
# This is so that the trained model is started fresh each time. However, this needs to be done before 
# tensorboard is started

#echo $OUTDIR $REGION $JOBNAME
os.environ["PROJECT"]



In [ ]:
%%bash
# Setup python so it sees the task module which controls the model.py
#RUNNING locally
export PYTHONPATH=${PYTHONPATH}:${PWD}/${MODEL_NAME}

python -m trainer.task

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
echo $PYTHONPATH

In [ ]:
%%bash
#running locally using ai platform
OUTDIR= ${PWD}/cloudmlemodels

In [ ]:
%%bash
#running locally using ai platform
#OUTDIR= ${PWD}/cloudmlemodels
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=${PWD}/trainer \
   -- \
   --train_dir=${PWD}/final_data/df_train.csv \
   --valid_dir=${PWD}/final_data/df_valid.csv  \
   --job-dir=${PWD}/cloudmlemodels \
   --OUTDIR=${PWD}/cloudmlemodels

In [ ]:
%%bash

OUTDIR=gs://${BUCKET}/final_model
echo $OUTDIR
#--staging-bucket=gs://$BUCKET \

In [ ]:
%%bash
#hptuning on cloudml
#make sure all data is in buckets.
OUTDIR=gs://${BUCKET}/${TRAINING_DIR}
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
gcloud ai-platform jobs submit training $JOBNAME \
   --config=hyperparam.yaml \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   --job-dir=$OUTDIR \
   --runtime-version=1.13 \
   --python-version 3.5 \
   --scale-tier=STANDARD_1 \
   --region=$REGION \
   --\
   --output_dir=$OUTDIR \

In [ ]:
%%bash
#Submitting job without hptuning
OUTDIR=gs://${BUCKET}/final_model2
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
gcloud ai-platform jobs submit training $JOBNAME \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   --job-dir=$OUTDIR \
   --runtime-version=1.13 \
   --python-version 3.5 \
   --scale-tier=STANDARD_1 \
   --region=$REGION \
   --\
   --output_dir=$OUTDIR \

In [ ]:
import pandas as pd
df_train=pd.read_csv("./final_data2/df_valid.csv")

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/my_local_model/cloudmlemodels/export/exporter | tail -1

In [ ]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/final_model2/export/exporter | tail -1)
echo $MODEL_LOCATION

In [ ]:
%%bash
echo $REGION

In [ ]:
%%bash
MODEL_NAME="virality_punjabkesri"
MODEL_VERSION="v4"#enter new version
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/my_local_model/cloudmlemodels/export/exporter | tail -1)
#echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ai-platform models create ${MODEL_NAME} --regions="asia-northeast1"
gcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} \
--origin ${MODEL_LOCATION} --runtime-version $TFVERSION --python-version=3.5


In [ ]:
#getting predictions from deployed model
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "virality_punjabkesri"
MODEL_VERSION = 'v4'
PROJECT = "tatvic-gcp-dev-team" 
token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
     {'0': 0.0035658744,
 '1': 0.004414475,
 '10': 0.0018055257,
 '11': 0.0014781109,
 '12': 0.0025544928,
 '13': 0.004745611,
 '14': 0.2856712,
 '2': 0.0020961564,
 '3': 0.0041580633,
 '4': 0.5815388,
 '5': 0.0034491895,
 '6': 0.002489877,
 '7': 0.0061961785,
 '8': 0.0018793311,
 '9': 0.09395714,
 'cat': 'sports',
 'hour': 0,
 'length': 302,
 'msaav10': 370.6,
 'num_of_identities': 4,
 'num_of_sentences': 3,
 'other_entities': 0,
 'polarity': 0.10714285714285714,
 'subjectivity': 0.5714285714285714,
 'weekday': 1,
 "upv3hr":55
     }]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

In [ ]:
print(response.content)

In [ ]:
#running locally
%%bash

python -m trainer.task --OUTDIR="./cloudmlemodels"